In [75]:
import os, copy, time, sys, random
import pandas as pd, numpy as np
from collections import deque
from datetime import datetime
import pickle
import warnings

import torch
import torch.nn as nn


# Normalization of observations:
from sklearn import preprocessing

import gym
from gym import spaces
import numpy as np

from utils.pipelines import preprocess_df, preprocess_price
from utils.split import time_split
# Custom Imports:
from render.graph import TradingGraph
from envs.to_gym import DiscreteGymEnvironment, ContiniousGymEnvironment
from utils.eval import eval_policy, evaluate
import preprocessing.transforms as T
from envs.discrete_stock_env import DiscreteStockEnvironment
from envs.continious_stock_env import ContiniousStockEnvironment
from utils.device import get_device
from agents.sb_agents import SBAgent

In [67]:
training_batch_size = 500
train_episodes = 1
visualize = False
env = ContiniousStockEnvironment(df = train_df, lookback_window_size = lookback_window_size, 
                               normalize_obs=True, initial_balance = 10000, sequential= False,
                               render_range = 300, reward_strategy='benchmark',
                               name = "Custom Trading Environment v1")


for episode in range(train_episodes):
    state = env.reset(n_iter = training_batch_size)

    for t in range(training_batch_size):
        env.render(visualize)
        action = env.action_space.sample()
        next_state, reward, done , info = env.step(action)
        state = next_state
        
        if done:
            state = env.reset(n_iter = training_batch_size)


In [69]:
device = torch.device(get_device())

In [72]:
#from stable_baselines3.common.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO


df = pd.read_csv('./data/BTCUSDT-1m-data_historical.csv')
df = preprocess_df(df)


lookback_window_size = 200 # 6 hours
test_window = 120 # Since 2021
train_df, test_df = time_split(df, lookback_window_size, test_window)



train_env = DummyVecEnv(
            [lambda: DiscreteStockEnvironment(df = train_df, lookback_window_size = lookback_window_size, 
                                              normalize_obs=True, initial_balance = 10000, sequential= False,
                                              render_range = 300, reward_strategy='base',
                                              name = "Custom Trading Environment v1")]
)


model = PPO('MlpPolicy', train_env, verbose=1,
            tensorboard_log="./runs/",
            learning_rate = 1e-5, clip_range = 0.3)



model.learn(total_timesteps= 30)





test_env = DummyVecEnv(
                [lambda: DiscreteStockEnvironment(df = test_df, lookback_window_size = lookback_window_size, 
                                                  normalize_obs=True, initial_balance = 10000, sequential= True,
                                                  render_range = 300, reward_strategy='base',
                                                  name = "Custom Trading Environment v1")]
)



Adding indicators...
Data is fetched from path!
Using cuda device
Logging to ./runs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 318  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 277          |
|    iterations           | 2            |
|    time_elapsed         | 14           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 9.028625e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.3          |
|    entropy_loss         | -3.4         |
|    explained_variance   | 8.01e-05     |
|    learning_rate        | 1e-05        |
|    loss                 | 2.65e+05     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000412    |
|    value_l

KeyboardInterrupt: 

In [265]:
import gym

from stable_baselines3 import A2C

env = gym.make("CartPole-v1")

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    if done:
      obs = env.reset()

env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 20.9        |
|    ep_rew_mean          | 20.9        |
| time/                   |             |
|    fps                  | 478         |
|    iterations           | 1           |
|    time_elapsed         | 4           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.008138352 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.593      |
|    explained_variance   | 0.568       |
|    learning_rate        | 0.0003      |
|    loss                 | 12.8        |
|    n_updates            | 50          |
|    policy_gradient_loss | -0.0154     |
|    value_loss           | 52.9        |
-----------------------------------------
-----------------------